# 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn

In [ ]:
sklearn.__version__

In [ ]:
train =  pd.read_csv('data/preprocess/train_0910.csv')
test =  pd.read_csv('data/preprocess/test_0910.csv')

In [ ]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

In [ ]:
#y

# RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split

# RMSLE 정의
def rmsle(origin, pred):
    return np.sqrt(mean_squared_log_error(origin, pred))

# X, y를 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)

### Train
- 우리가 가진 데이터셋 내에서 훈련 -> 예측 -> 검증을 해보면서 모델의 성능을 평가/향상시키려고 함
- 그래서 가격을 알고 있는 데이터(여기선 X,y)를 7:3 비율로 train : test로 쪼개서 우리가 돌리는 모델을 검증해보려고 함
- 근데 아직은 K-Fold Validation이라던가 유의미한 방법은 적용 안해봄.  

그래서 아직은
- 큰 의미가 있는 과정이라기보다는, RandomForestRegressor가 잘 돌아가는구나~
- 우리가 가진 데이터 내에서 검증해봤을때 RMSLE 가 0.012면 꽤 잘 나오는구나~
- feature importance 관찰용
- 정도로 생각하시면 될 듯 (이제 여러가지 시도해봐야죠 ㅎㅎ)

In [ ]:
forest_train = RandomForestRegressor(n_estimators=1000, 
                              criterion='mse',
                              random_state=1,
                              n_jobs=-1)
forest_train.fit(X_train, y_train)
y_train_pred = forest_train.predict(X_train)
y_test_pred = forest_train.predict(X_test)

print('훈련 MSE: %.3f, 테스트 MSE: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('훈련 R^2: %.3f, 테스트 R^2: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))
print('훈련 RMSLE: %.3f, 테스트 RMSLE: %.3f' % (
        rmsle(y_train, y_train_pred),
        rmsle(y_test, y_test_pred)))

importance가 낮은 feature drop

In [ ]:
importance = forest_train.feature_importances_

In [ ]:
del_list = []
for i in range(len(test.columns.tolist())):
    if importance[i] < 0.0001:
        del_list.append(test.columns.tolist()[i])
len(del_list)

In [ ]:
X = X.drop(columns=del_list,axis=1)
test = test.drop(columns=del_list,axis=1)

In [ ]:
forest = RandomForestRegressor(n_estimators=1000, 
                              criterion='mse',
                              random_state=1,
                              n_jobs=-1)
forest.fit(X, y)
pred = forest.predict(test)
pred = np.expm1(pred)

In [ ]:
pred

- 저장

In [ ]:
# sub=pd.read_csv('data/sample_submission.csv')
# sub['SalePrice']=pred
# sub.to_csv('submission/sub_1_0910.csv',index=False)

# XGBoost
- 이건 아직 복붙해서 돌려본거라...
- 대충 돌려봤는데도 RandomForest보다 훨씬 잘 나옴
- 더 잘 해보고 수정하겠음...

In [ ]:
# from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
# from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import RobustScaler
# from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
# from sklearn.metrics import mean_squared_error
# from xgboost import XGBRegressor
# import xgboost as xgb
# import lightgbm as lgb
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression
# from sklearn.model_selection import GridSearchCV

In [ ]:
# #Validation function
# n_folds = 10

# def rmsle_cv(model):
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
#     rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
#     return(rmse)

In [ ]:
# model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213, silent=1,
#                              random_state =7, nthread = -1)

In [ ]:
# model_xgb.fit(X_train, y_train)
# xgb_train_pred = model_xgb.predict(X_train)
# xgb_pred = np.expm1(model_xgb.predict(X_test))
# print(rmsle(y_train, xgb_train_pred))
# print(rmsle(y_test, xgb_pred))

In [ ]:
# model_xgb.fit(X, y)
# pred_xgb = model_xgb.predict(test)

In [ ]:
# pred_log_xgb = np.expm1(pred_xgb)

In [ ]:
# sub=pd.read_csv('data/sample_submission.csv')
# sub['SalePrice']=pred_log_xgb
# sub.to_csv('sub_0910_xgb.csv',index=False)

# 앙상블